In [1]:
import numpy as np
from collections import namedtuple
from IPython.core.debugger import Pdb

from GameEngine import multiplayer
Point = namedtuple('Point', 'x, y')
SA = namedtuple('SA', 'state, action')

pygame 2.5.1 (SDL 2.28.2, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


### New Game Implementation

I have an improved game implementation which allows for multiplayer snake games, as well as simplified training. This notebook will go over both of these, including an implementation of q-table learning, as well as a match between a manually filled out q-table and a learned one.

I'll start by initializing the engine object:

In [2]:
# defines game window size and block size, in pixels
WINDOW_WIDTH = 640
WINDOW_HEIGHT = 480
GAME_UNITS = 80

In [3]:
game_engine = multiplayer.Playfield(window_width=WINDOW_WIDTH,
                                    window_height=WINDOW_HEIGHT,
                                    units=GAME_UNITS,
                                    g_speed=35,
                                    s_size=1)

Here is a run-down of the current functions available to programs utilizing the game engine:

**add_player**: Returns the player's number to the callee (between 0-3, for a total of 4 players). This number can be used with other functions to index that player's state.

**get_heads_tails_and_goal**: Returns an array of player heads (in order of player number) the locations of all snake tails, as well as the goal location. Each is stored in an array of named tuples.

**get_viable_actions**: Given a player's id, returns a list of integers corresponding to actions which will not immediately result in the snake's death.
0 = UP
1 = RIGHT
2 = DOWN
3 = LEFT

**start_game**: Initializes goal, player, score, and playfield objects. Disables the ability to add new players. Enables use of player_advance function.

**stop_game**: Sets the game_state to false, allowing new players to be added.

**cleanup**: Quits the pygame window.

**player_advance**: Given an array corresponding to each player's action (integers), returns a list of collision results, and updates the internal game state.

**toggle_draw**: Turns off/on the game UI for faster training.

I can do a test of these functions:

In [4]:
p1 = game_engine.add_player()
game_engine.start_game()
p1

Game starting with 1 players.


0

In [5]:
viable_actions = game_engine.get_viable_actions(p1)

In [6]:
game_engine.player_advance([np.random.choice(viable_actions)])

[<CollisionType.NONE: 2>]

If you looked at the UI for this last statement, you should have seen that the game moved the snake (yellow) in a random direction away from immediate death.

### State-sensing methods, creating and reading a q-table
Now, we can start redesigning some functions used to allow the snake to play intelligently. We'll use a multi-dimensional numpy array to store the rewards corresponding to each state and action. This is called a q-function, or a q-table in this case.

How many states do I need? Seeing how the new **get_viable_actions** method already prevents the snake from choosing life-ending moves, the snake is no longer tasked with learning or memorizing it.

The snake does need to be able to interpret progress towards the goal, so I will reinclude one state; the compass direction from the snake's head to the goal. This means I need only 8 states (with entries for each four actions) to represent my game now.

In [7]:
goal_relations = 8
actions = 4
q = np.zeros((goal_relations,
              actions))
q

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [8]:
def sense_goal(head, goal):
    '''
    maps head and goal location onto an
    integer corresponding to approx location
    '''
    diffs = Point(goal.x - head.x, goal.y - head.y)

    if diffs.x == 0 and diffs.y <  0:
        return 0
    if diffs.x >  0 and diffs.y <  0:
        return 1
    if diffs.x >  0 and diffs.y == 0:
        return 2
    if diffs.x >  0 and diffs.y >  0:
        return 3
    if diffs.x == 0 and diffs.y >  0:
        return 4
    if diffs.x <  0 and diffs.y >  0:
        return 5
    if diffs.x <  0 and diffs.y == 0:
        return 6
    return 7

I will use the getter provided by my engine, which queries various statistics about all agents in the game:
1. An array of head positions
2. An array of all tail locations
3. The goal location

In [9]:
game_engine.get_heads_tails_and_goal()

([Point(x=320, y=160)], [Point(x=320, y=160)], Point(x=0, y=400))

Now to make a function to index our expected reward-to-go given a state using sense_goal:

In [10]:
def index_actions(q, id):
    '''
    given q, player_id, an array of heads,
    and the goal position,
    indexes into the corresponding expected
    reward of each action
    '''
    heads, tails, goal = game_engine.get_heads_tails_and_goal()
    state = sense_goal(heads[id], goal)
    return state, q[state, :]

Returning state here simplifies some logic later when I train the agent. It will be passed along to my next function, but it can be ignored for now.

In [11]:
_, rewards = index_actions(q, p1)
rewards

array([0., 0., 0., 0.])

In our learning agent, these actions will obviously be associated with different expected rewards. Essentially, we have a function that, given a state, tells us the expected utility of each action. Should we just choose the best one?

There are two problems with a greedy approach...

1. If the agent only sticks to the one policy it knows, then it will never truly learn the best solution. This is not the largest problem, due to how few states we have. Additionally, the snake is forced into a new state everytime it collects a goal. Still, a pure greedy approach results in slow, sub-optimal training.
2. My snake is ignorant of which actions will result in a collision. The snake must understand that its Q function will commonly lead it astray in navigating its environment.

In cases where the snake has explored enough to make its Q function useful and death is not a possibility, I still do want the snake to be greedy. I chose to implement a replacement argmin/max function to select actions from this table, which generates new actions in order from highest expected reward to lowest expected reward.

In [12]:
def argmin_gen(rewards):
    rewards = rewards.copy()
    for i in range(rewards.size):
        best_action = np.argmin(rewards)
        rewards[best_action] = float("inf")
        yield best_action

In [13]:
for action in argmin_gen(rewards):
    print(action)

0
1
2
3


How will I use this? Remember that my game engine includes all the logic to calculate which actions are immediately dangerous. If the action generated is not a viable action, we will take the next best action, or the next, next best action, etc.

What if no actions are viable? Then the agent has boxed itself in, and it doesn't matter what action we choose.

Previously, I reset the snake if it got stuck in a learning-loop. I will instead use epsilon, as it is the tried-and-true method of enforcing exploration.

Here is my greedy-epsilon function, combining the work of all the previous code:

In [14]:
def pick_greedy_action(q, id, epsilon):
    viable_actions = game_engine.get_viable_actions(id)
    state, rewards = index_actions(q, id)

    if np.random.uniform() < epsilon:
        # SA -- a STATE-ACTION pair (X as in function input)
        return SA(state, np.random.choice(viable_actions)) if viable_actions.size > 0 else SA(state, 0)
    for action in argmin_gen(rewards):
        if action in viable_actions:
            return SA(state, action)
    return SA(state, 0) # death

I'll set up epsilon to decay over our 500-step test...

In [15]:
n_steps = 200
epsilon = 1
final_epsilon = 0.001
epsilon_decay =  np.exp(np.log(final_epsilon) / (n_steps))

And watch the snake explore:

In [16]:
for step in range(n_steps):
    _, p1_action = pick_greedy_action(q, p1, epsilon)
    game_engine.player_advance([p1_action])
    epsilon *= epsilon_decay

This snake obviously has no prior knowledge of how to earn the most reward, but it still does remarkably well because it is not allowed to die. It behaves as expected, favoring up and right when it is not forced to choose a random action.

Our q_table only has 32 values as a result of removing the 16 danger states... It would be incredibly easy to manually select reward values to fill our q_table with...

In [17]:
set_q = np.array([[-10., -2., 0., -2.],
                  [-5., -5., 0., 0.],
                  [-2., -10., 2., 0.],
                  [0., -5., -5., 0.],
                  [0., -2., -10., -2.],
                  [0., 0., -5., -5.],
                  [-2., 0., -2., -10.],
                  [-5., 0., 0., -5.]])

In [18]:
epsilon = 0
for step in range(n_steps):
    X = pick_greedy_action(set_q, p1, epsilon)
    game_engine.player_advance([X.action])
    epsilon *= epsilon_decay

And the snake already plays optimally, no learning required. This implementation might be more similar to a passive learning agent, in the sense I already told the snake what policy I want it to follow.

Now that we have these methods, I will create functions to allow the snake to learn by its own, and then pair it off against the q-table I just built.

### Q-Learning, with Temporal Difference

A good agent prioritizes actions that leads to the highest expected reward. The Q-function assigns an expected utility to each state-action pair, usually the expected reward-to-go.

A popular method of adjusting this state-value function is a version of the temporal difference equation, which adjusts the utility associated with each input to agree with the maximum utility of its successor:

$Q(s,a) = Q(s,a) + \alpha[(R(s,a,s') + \gamma * max _a'Q(s',a') - Q(s,a))]$

The nature of this method is somewhat recursive, as it updates Q to agree with max(Q'), which in turn is updated to agree with max(Q'')... which is why the algorithm is sometimes referred to as bootstrapping.

The discount factor $\gamma$ can be used to weight immediate reward higher than future reward, though will be kept as 1 in my solution, which means we consider all future actions equally. All I need to do is assign an enticing enough reinforcement to goal-attaining actions, and use the temporal difference equation to update all other state transitions.

In order to implement this equation, I simply need a function that takes the q-table to be updated, the old state-action pair, and then ew state-action pair, and the outcome as returned by the game engine so we can assign a reward.

When the agent does reach the goal, I will manually set that state and action to the best reward, 0. Remember that the q-table is initialized with zeros, meaning untravelled actions are pre-assigned good rewards. Both this and epsilon will encourage exploration.

Here is the complete function:

In [19]:
reward = -1

def update_q(q, old_X, new_X, outcome, lr=0.05):
    if outcome == multiplayer.CollisionType.GOAL:
        q[new_X.state, new_X.action] = 0
    else:
        td_error = reward + q[new_X.state, new_X.action] - q[old_X.state, old_X.action]
        q[old_X.state, old_X.action] += lr * td_error

Now all that is needed is the training loop. I have high expectations for this agent, so I will only allow it 1500 moves to train itself! Here is where the outputs of pick_greedy_action come in handy, because they can be used as a direct index into Q:

In [20]:
n_steps = 1500
epsilon = 1
final_epsilon = 0.001
epsilon_decay =  np.exp(np.log(final_epsilon) / (n_steps))

In [21]:
p1_old_X = pick_greedy_action(q, p1, epsilon) # state, action
game_engine.player_advance([p1_old_X.action])

for step in range(n_steps):
    p1_new_X = pick_greedy_action(q, p1, epsilon) # state, action
    outcome = game_engine.player_advance([p1_new_X.action])

    update_q(q, p1_old_X, p1_new_X, outcome)

    epsilon *= epsilon_decay
    p1_old_X = p1_new_X

Most of the time, results look promising. Here is everything it learned:

In [22]:
q

array([[-2.35070513, -1.09149338, -0.16535442, -0.56686264],
       [-9.89365379, -3.92409428, -5.34681609, -1.89000207],
       [-0.11260587, -4.74939046, -9.9192565 , -0.43756285],
       [-1.40606955, -3.09973297, -1.00213181, -0.59918486],
       [ 0.        , -0.48427898, -3.18961465, -0.15631095],
       [-1.23097846, -1.83318991, -3.93160124, -1.29734622],
       [-0.85699409, -0.19789849, -0.82954074, -5.23579225],
       [-4.3355246 , -1.17885939, -0.53698344, -2.4048526 ]])

### Multiplayer Demonstration, Saving Tables

The most entertaining way to test the success of my implementation is pair the agents q and set_q against each other. I will first stop and set up a new game:

In [23]:
game_engine.stop_game()
p2 = game_engine.add_player()
game_engine.start_game()

Game over!
Game starting with 2 players.


Now, I can simply call player advance with both player's actions in order, and the engine will handle the rest. I will define a new game loop, similar to the previous one:

In [24]:
epsilon = 0

In [25]:
for step in range(n_steps):
    # p1
    p1_X = pick_greedy_action(set_q, p1, epsilon)

    # p2
    p2_X = pick_greedy_action(q, p2, epsilon) # state, action
    
    game_engine.player_advance([p1_X.action, p2_X.action])

    epsilon *= epsilon_decay

The learned agent usually plays almost as well as the artificially-learned one, which is okay given I hardly spent time optimizing the number of steps and learning rate. I plan to compare both of the agents again my neural-network approach, so the last I will do is save the q_tables to a file.

In [26]:
np.save('superior_qt.npy', set_q)
np.save('inferior_qt.npy', q)